# SageMaker で scikit-learn コンテナを使った学習・推論を行う

#### ノートブックに含まれる内容

- scikit-learn を SageMaker で行うときの，基本的なやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: DecisionTreeClassifier
- データ: iris

## セットアップ

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

## 学習用データを S3 にアップロード

SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，ローカルにある iris データをいったん SageMaker SDK の `session` クラスにある `upload_data()` メソッドを使って，ノートブックインスタンスのローカルから S3 にアップロードします．

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．`upload_data()` メソッドの引数に `bucket=XXXX` という形でデータを配置するバケットを指定することが可能です．

以下を実行する前に，**<span style="color: red;">5 行目の `data/scikit-byo-iris/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
import numpy as np
import os
from sklearn import datasets

PREFIX = 'data/scikit-iris/XX'

# Load Iris dataset, then join labels and features
iris = datasets.load_iris()
joined_iris = np.insert(iris.data, 0, iris.target, axis=1)

# Create directory and write csv
os.makedirs('./data', exist_ok=True)
np.savetxt('./data/iris.csv', joined_iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')

train_input = sagemaker_session.upload_data('data', key_prefix=PREFIX)

## モデルの学習を実行

SageMaker で学習を行うために，SageMaker SDK で `Estimator` オブジェクトをつくります．このオブジェクトには，学習をおこなうために以下の設定が含まれます．その上で，`fit()` メソッドで学習を実施します．学習には 5 分程度時間がかかります．

* __role__: ジョブを実行する IAM role
* __instance count__:  学習ジョブに使うインスタンス数
* __instance type__ 学習ジョブに使うインスタンスタイプ
* __output path__: 学習の成果物が置かれる S3 の場所
* __session__: すぐ上で作成した，SageMaker セッション

scikit-learn コンテナを使う場合，基本的にはスクリプトの `__main__` 関数内に，学習処理をベタ書きすれば OK です．その際に，入力データの場所やモデルファイルを出力する場所などは，環境変数として SageMaker から引き渡されます．具体的な環境変数の一覧は以下のとおりです．

* `SM_MODEL_DIR`: 出力モデルを配置する，コンテナ内のディレクトリのパスをさします．このパスにモデルファイルを出力しておけば，SageMaker が学習終了時にフォルダの中身を tar.gz にまとめて，S3 に出力してくれます．
* `SM_OUTPUT_DIR`: モデルファイル以外の出力ファイルを置くためのディレクトリパスです．こちらも同様に，SageMaker が S3 にデータを出力します．
* `SM_CHANNEL_TRAIN`: `fit()` を実行する際に指定するデータのうち，`train` タグのついた学習用データが置かれる，コンテナ内のディレクトリパスをさします
* `SM_CHANNEL_TEST`: 上と同様に，`test` タグのついた検証用データのパスをさします．

また `Estimator` オブジェクト作成時に，Hyperparameter として指定したものは，引数としてスクリプトに渡ってくるので，argparse パッケージを用いて取得可能です．
こちらは `scilit_learn_iris.py` をご覧ください．

もし追加のモジュールインストールが必要な場合には，`source_dir` 直下に `requirements.txt` を配置することで，コンテナ起動時にインストールされます．

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'scikit_learn_iris.py'
source_dir='src/'

sklearn = SKLearn(
    entry_point=script_path,
    source_dir=source_dir,
    train_instance_type="ml.m4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={'max_leaf_nodes': 10})

sklearn.fit({'train': train_input})

## モデルの推論を実行

推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．また併せて，オプションで（リクエストで渡されるデータの）シリアライザと（レスポンスで返されるデータの）デシリアライザを指定することも可能です．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

In [ ]:
import pandas as pd

# 推論用のデータを準備
test_X = pd.DataFrame([[5.0, 3.2, 1.2, 4.3], [4.5, 2.3, 1.3, 0.3], [5.7, 2.8, 4.1, 1.3]])

print(test_X)

In [ ]:
# 推論を実行して，結果を表示
result = predictor.predict(test_X.values)
print(result)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sklearn.delete_endpoint()